In [2]:

import numpy as np
from scipy import optimize 
import pandas as pd

def MaximizeSharpeRatioOptmzn(MeanRet, CovarRet, RiskFreeRate, Port_Size):
    """
    Maximize the Sharpe ratio using the principle of duality.

    Parameters:
    - MeanRet: Mean returns of assets.
    - CovarRet Covariance matrix of returns.
    - RiskFreeRate: Risk-free rate.
    - Port_Size: Number of assets in the portfolio.

    Returns:
    - scipy.optimize.OptimizeResult: Result of the optimization.
    """
    # defining maximum Sharpe Ratio 
    def  max_sharpe(x, MeanRet, CovarRet, RiskFreeRate, Port_Size):
        Denom = np.sqrt(np.matmul(np.matmul(x, CovarRet), x.T) )
        Numer = np.matmul(np.array(MeanRet),x.T)-RiskFreeRate
        sharpe_rat = -(Numer / Denom)
        return sharpe_rat

    #define equality constraint representing fully invested portfolio
    def const_Eq(x):
        A=np.ones(x.shape)
        b=1
        constraintVal = np.matmul(A,x.T)-b 
        return constraintVal
        

    #defining parameters inculding bounds
    xinit=np.repeat(0.33, Port_Size)
    cons = ({'type': 'eq', 'fun':const_Eq})
    lb = 0 
    ub = 1 
    bounds = tuple([(lb,ub) for x in xinit])
    
    #using scipy optimiser to for portfolio optimisation
    opt = optimize.minimize (f, x0 = xinit, args = (MeanRet, CovarRet,\
                             RiskFreeRate, Portf_Size), method = 'SLSQP',  \
                             bounds = bounds, constraints = cons, tol = 10**-3)
    
    return opt
    

In [3]:
 
def StockReturnsComputing(StockPrice, Rows, Columns):
    """
    Compute the stock returns based on daily stock prices.

    Parameters:
    - StockPrice: Numpy array of daily stock prices.
    - Rows: Number of rows in the array.
    - Columns: Number of columns in the array.

    Returns:
    - Numpy array: Stock returns.
    """
    import numpy as np
    
    StockRet = np.zeros([Rows-1, Columns])
    for j in range(Columns):        # j: stocks
        for i in range(Rows-1):     # i: Daily Clsoing Prices
            StockRet[i,j]=((StockPrice[i+1, j]-StockPrice[i,j])/StockPrice[i,j])* 100

    return StockReturn

In [115]:
# Read data from CSV file
df = pd.read_csv('test_tt_10.csv')

Rows = df.shape[0]
Columns = len(df.columns)


#extract stock ticker labels
assetLabels = df.columns[0:Columns+1].tolist()
print('Asset labels of portfolio: \n', assetLabels)

#read closing prices data
StockData = df.iloc[0:, 0:]

#compute asset returns
arStockPrices = np.asarray(StockData)
[Rows, Cols]=arStockPrices.shape
arReturns = StockReturnsComputing(arStockPrices, Rows, Cols)
np.set_printoptions(precision=3, suppress = True)

#compute mean returns and variance covariance matrix of returns using numpy methods
meanReturns = np.mean(arReturns, axis = 0)
covReturns = np.cov(arReturns, rowvar=False)
print('\nMean Returns:\n', meanReturns)
print('\nVariance-Covariance Matrix of Returns:\n', covReturns)

685
10
Asset labels of k-portfolio 1: 
 ['HSBA.L', 'AON', 'BA.L', 'LLOY.L', 'BATS.L', 'RIO.L', 'NG.L', 'RKT.L', 'AAL.L', 'SHEL.L']
       HSBA.L         AON        BA.L     LLOY.L  BATS.L   RIO.L        NG.L   
0  423.750000  232.529999  495.500000  38.880001  2536.5  6436.0  828.200012  \
1  432.899994  228.009995  499.100006  40.415001  2555.0  6435.0  810.200012   
2  426.500000  226.039993  497.399994  40.345001  2589.5  5878.0  829.200012   
3  437.600006  232.759995  483.500000  40.075001  2614.5  5854.0  828.000000   
4  455.500000  230.000000  496.399994  41.779999  2607.5  6040.0  831.599976   

    RKT.L        AAL.L       SHEL.L  
0  6118.0  2943.869873  1470.199951  
1  6100.0  2971.243164  1490.400024  
2  6282.0  2873.694580  1522.800049  
3  6270.0  2900.072510  1548.400024  
4  6210.0  3004.588623  1546.000000  

Mean Returns:
 [ 0.071  0.044 14.244  0.047 -0.006  0.005 14.479 -0.009 -0.026  0.097]

Variance-Covariance Matrix of Returns:
 [[     2.631      0.248     26.

In [116]:

#set portfolio size
port_Size = Columns

#set risk free asset rate of return value as 3
Rf=3 
annRiskFreeRate = Rf/100

#compute daily risk free rate in percentage
r0 = (np.power((1 + annRiskFreeRate),  (1.0 / 360.0)) - 1.0) * 100 
print('\nRisk free rate (daily %): ', end="")
print ("{0:.3f}".format(r0)) 

#initializating the variables
xOptimal =[]
minRiskPoint = []
expPortfolioReturnPoint =[]
maxSharpeRatio = 0

#computing maximal Sharpe Ratio and optimal weights
result = MaximizeSharpeRatioOptmzn(meanReturns, covReturns, r0, portfolioSize)
xOptimal.append(result.x)

    
#compute risk returns and max Sharpe Ratio of the optimal portfolio   
xOptimalArray = np.array(xOptimal)
Risk = np.matmul((np.matmul(xOptimalArray,covReturns)), np.transpose(xOptimalArray))
expReturn = np.matmul(np.array(meanReturns),xOptimalArray.T)
annRisk =   np.sqrt(Risk*251) 
annRet = 251*np.array(expReturn) 
maxSharpeRatio = (annRet-Rf)/annRisk 
np.set_printoptions(precision=3, suppress = True)


#display results
print('Maximal Sharpe Ratio: ', maxSharpeRatio, '\nAnnualized Risk (%):  ', \
      annRisk, '\nAnnualized Expected Portfolio Return(%):  ', annRet)
print('\nOptimal weights (%):\n',  xOptimalArray.T*100 )


Risk free rate (daily %): 0.008
Maximal Sharpe Ratio:  [[0.605]] 
Annualized Risk (%):   [[1191.106]] 
Annualized Expected Portfolio Return(%):   [723.828]

Optimal weights (%):
 [[10.026]
 [10.016]
 [ 9.979]
 [10.017]
 [ 9.996]
 [10.   ]
 [ 9.946]
 [ 9.996]
 [ 9.987]
 [10.039]]
